In [1]:
# import web driver
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
import time
import random
import re
from seleniumrequests import Chrome
import getpass
import os
import pandas as pd
import datetime as dt
from bs4 import BeautifulSoup

Prepare Chicago Crime Data

In [2]:
# # Preprocessing Chicago Crime Data

# # Import Chicago Crime Data : Source : https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2
# df_crime = pd.read_csv("chicago_crime.csv")

# # Convert text date to datetime
# df_crime['DATE_TIME'] = pd.to_datetime(df_crime['Date'],format="%m/%d/%Y %I:%M:%S %p", errors = 'coerce')

# # Strip hr, min, sec data so that aggregation per day is possible
# df_crime['DATE_TIME_TRUN'] = df_crime['DATE_TIME'].map(lambda x: x.strftime('%Y-%m-%d'))

# # Drop unneeded columns
# df_crime = df_crime[['ID','DATE_TIME_TRUN']].copy()

# # Just get the date and total crime count and create new df after aggregating by single day
# df_crime = df_crime.groupby(['DATE_TIME_TRUN'],as_index = False).count().copy()

# # Save cleaned data
# df_crime.to_csv('chicago_crime_cleaned_aggregated.csv',index=False)

In [3]:
df_crime = pd.read_csv('chicago_crime_cleaned_aggregated.csv')
df_crime['DATE_TIME'] = pd.to_datetime(df_crime['DATE_TIME_TRUN'])
df_crime['YEAR_MONTH'] = df_crime['DATE_TIME_TRUN'].apply(lambda x:x[:-3])
df_crime = df_crime[df_crime['DATE_TIME_TRUN']>'2009-09-01'] #limited by weather data

Weather Data Scrape

In [40]:
# Initiate Selenium Chrome driver for Mac
git_folder_location = os.path.abspath(os.path.dirname('metis_proj_2_luther'))
full_path_to_chromedriver = os.path.join(git_folder_location, "chromedriver")
driver = Chrome(executable_path = full_path_to_chromedriver)

In [145]:
year_month_list = list(df_crime['YEAR_MONTH'].unique())



def scrape_weather_url(url):
    driver.get(url)
    #response = driver.page_source
    time.sleep(6)
    days_chain = [x.find_all('a') for x in soup.find_all(class_='weatherLinks')]
    for ix,link in enumerate(days_chain[0]):
        
        '''
        Bottom section tries to solve loading issue
        Refer : https://selenium-python.readthedocs.io/waits.html
        '''
        wait.until(ExpectedConditions.stalenessOf("//div[@class='weatherLinks']/a[{}]".format(ix+1)))
        day_link = driver.find_element_by_xpath("//div[@class='weatherLinks']/a[{}]".format(ix+1))
        day_link.click()
        wait.until(ExpectedConditions.stalenessOf("//div[@class='temp']")[-1].text)
        high_low.append(driver.find_elements_by_xpath("//div[@class='temp']")[-1].text) #notice elements, s at the end. This returns a list, and I can index it.
#               
        
        
#         scraped = False
#         while not scraped:
#             try:
#                 day_link = driver.find_element_by_xpath("//div[@class='weatherLinks']/a[{}]".format(ix+1))
#                 day_link.click()
#                 scraped = True
#             except:
#                 pass     

#         time.sleep(1)
#         fully_loaded = False
#         while not fully_loaded:
#             try:
#                 high_low.append(driver.find_elements_by_xpath("//div[@class='temp']")[-1].text) #notice elements, s at the end. This returns a list, and I can index it.
#                 fully_loaded = True
#             except:
#                 pass 
        


        # Selenium locating element document : https://selenium-python.readthedocs.io/locating-elements.html#locating-elements
        weather_desc.append(driver.find_element_by_xpath("//div[@class='wdesc']").text)
        humidity_barometer.append(driver.find_element_by_xpath("//div[@class='mid__block']").text)
        wind.append(driver.find_element_by_xpath("//div[@class='right__block']").text)
        date_time.append(driver.find_elements_by_xpath("//div[@class='date']")[-1].text)

        
high_low, weather_desc, humidity_barometer, wind, date_time = [], [], [], [], []


for ix,date in enumerate(year_month_list[0:5]):
    month = int(year_month_list[ix][5:])
    year = year_month_list[ix][:4]
    url = 'https://www.timeanddate.com/weather/usa/chicago/historic?month={}&year={}'.format(month,year)
    scrape_weather_url(url)

NameError: name 'wait' is not defined

### Fine Next 
soup.find('a').findNextSibling()
### Return a list of all matches
soup.find_all('a')  
[link for link in soup.find_all('a') if 'joelcoen' in str(link)]
### Retrieve the url from an anchor tag
soup.find('a')['href']
### Find all based on id or class
soup.find_all(id='top_links')  
soup.find_all(class_='mp_box_content')
### Beautiful Soup - Chaining Finds
chain = [x.find_all('td') for x in soup.find_all(class_='mp_box_content')]  

To extract just the value of interest:   
soup.find(class_='mp_box_content').find_all('td')[1].text

In [86]:
# specifies the path to the chromedriver.exe
#driver = webdriver.Chrome('chromedriver.exe')

# # For PC
# driver = Chrome()

# For Mac
git_folder_location = os.path.abspath(os.path.dirname('metis_proj_2_luther'))
full_path_to_chromedriver = os.path.join(git_folder_location, "chromedriver")
driver = Chrome(executable_path = full_path_to_chromedriver)

# driver.get method() will navigate to a page given by the URL address
driver.get('https://www.linkedin.com/login?trk=guest_homepage-basic_nav-header-signin')

# Sleep for Random Seconds Between 1 and 3
time.sleep(random.randint(10,30)/10)

# Enter ID
username = driver.find_element_by_name("session_key")
username_input = getpass.getpass("LinkedIn Username: ")
username.send_keys(username_input)

# Sleep for Random Seconds Between 1 and 3
time.sleep(random.randint(10,30)/10)

# Enter Password
password = driver.find_element_by_name("session_password")
password_input = getpass.getpass("LinkedIn Password: ")
password.send_keys(password_input)

# Sleep for Random Seconds Between 1 and 3
time.sleep(random.randint(10,30)/10)

# Click Submit Button
submit = driver.find_element_by_xpath('//*[@type="submit"]')
submit.click()

# Sleep for Random Seconds Between 1 and 3
time.sleep(random.randint(10,30)/10)

# Navigate to Metis Alumni page
driver.get('https://www.linkedin.com/school/metis/people/?facetGeoRegion=us%3A0')

# Sleep for Random Seconds Between 1 and 3
time.sleep(random.randint(10,30)/10)

# Load Entire Page by Scrolling to the Bottom
SCROLL_PAUSE_TIME = 2# Scroll to Very Bottom to Load All
last_height = driver.execute_script("return document.body.scrollHeight")# Get scroll height
while True:    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # Scroll down to bottom
    time.sleep(SCROLL_PAUSE_TIME) # Wait to load page    
    new_height = driver.execute_script("return document.body.scrollHeight")# Calculate new scroll height and compare with last scroll height
    if new_height == last_height:
        break
    last_height = new_height

LinkedIn Username: ········
LinkedIn Password: ········


In [33]:
# Load source code of fully loaded page
page_source = driver.page_source

# Obtain LinkedIn url of All Alumni whose profiles are public
linkedin_url_list_dirty = []
for m in re.finditer('people_profile_card_name_link',page_source):
    linkedin_url_list_dirty.append(page_source[m.end()+8:m.end()+60])
linkedin_url_list = [url.split()[0][:-1] for url in linkedin_url_list_dirty]

In [34]:
# Total parsed LinkedIn addresses of Metis Alumni
total_num_alumni = page_source.count('artdeco-entity-lockup--stacked-center')
total_num_alumni_public = page_source.count('people_profile_card_name_link')
print("There are {} Metis Alumni, but {} of their profiles are private.  Hence, only {}'s linkedin addresses were parsed.".format(total_num_alumni,total_num_alumni-total_num_alumni_public,total_num_alumni_public))

# # Example of parsed LinkedIn addresses
# print("Example of parsed LinkedIn addresses :")
# linkedin_url_list[:5]

There are 151 Metis Alumni, but 6 of their profiles are private.  Hence, only 145's linkedin addresses were parsed.


In [49]:
# Test list
#linkedin_url_list_temp = ["/in/matthew-eungoo-lee/"]
linkedin_url_list_temp = [linkedin_url_list[13]]

# Function to Scrape data off of LinkedIn profile
def scrape_linkedin_profile(url):
    # Keep using same tab for each url
    driver.get(url)
    
#     # Create new tap for each url. This block doesn't work. It's not really needed for project though.
#     print(url)
#     script_text = "'''window.open('"+url+ "','_blank');'''"
#     driver.execute_script(script_text)

    # Sleep for Random Seconds Between 1 and 3
    time.sleep(random.randint(10,30)/10)
    
    
# Execute scrape_linkedin_profile function for each profile
for url in linkedin_url_list_temp: # Replace 'linkedin_url_list_temp' with 'linkedin_url_list' after testing
    scrape_linkedin_profile('https://www.linkedin.com'+url)
    

In [133]:
# # Opening in new tabs. Below two codes work.
# driver.execute_script('''window.open("https://www.linkedin.com/in/matthew-eungoo-lee/",'_blank');''')
# driver.execute_script('''window.open("https://www.google.com","_blank");''')

**Tip from Joe**

Here's a short summary of how you would likely develop and structure scraping code:
1. Starting with one page, write parsing functions that should generalize to any page with the same format.
2. Test your code on several different pages to confirm that it works in general. Try to make your code robust by checking returns for None and using try;except clauses to accommodate missing elements like in the get_movie_value function above.
3. Figure out the total collection of webpages that you want to scrape, and collect the urls into a list (e.g. 2018 movies 1-100, 101-200, etc.). Iterating through all the urls, request and parse the page, adding the data to a list of dicts.
4. Incrementally convert the list of dicts into a dataframe and save it to disk with pd.to_csv().
Note that this workflow leaves out some details you may need like intentional pausing, but we'll get to these soon :) 
Also, this is a suggestion, not a prescription. You should think of project luther as a first foray into setting up a data acquisition pipeline, and it takes some intelligent and creative design to get it right. Our best advice is to start scraping early and see where things break quickly so that you can fix it! 

[LinkedIn Scrape Data](https://www.linkedin.com/pulse/how-easy-scraping-data-from-linkedin-profiles-david-craven/)  
[Metis Alumni LinkedIn](https://www.linkedin.com/school/metis/people/?facetGeoRegion=us%3A0)



## HTML to BeautifulSoup

### Request data for The Big Lebowski

Scrape some information about [The Big Lebowski](http://boxofficemojo.com/movies/?id=biglebowski.htm).

In [4]:
from __future__ import print_function, division

In [5]:
# if needed: pip install requests or conda install requests
import requests

requests.__path__

['C:\\Users\\matt\\Anaconda3\\envs\\metis\\lib\\site-packages\\requests']

In [6]:
url = 'http://boxofficemojo.com/movies/?id=biglebowski.htm'

response = requests.get(url)

### Check the Status

For information on HTTP status codes, see:

https://en.wikipedia.org/wiki/List_of_HTTP_status_codes

In [7]:
response.status_code # status code = 200 => OK

200

### Look at the Text

In [7]:
print(response.text)

<html><head>
<script type="text/javascript">
window.onload = function() {
  // Parse the tracking code from cookies.
  var trk = "bf";
  var trkInfo = "bf";
  var cookies = document.cookie.split("; ");
  for (var i = 0; i < cookies.length; ++i) {
    if ((cookies[i].indexOf("trkCode=") == 0) && (cookies[i].length > 8)) {
      trk = cookies[i].substring(8);
    }
    else if ((cookies[i].indexOf("trkInfo=") == 0) && (cookies[i].length > 8)) {
      trkInfo = cookies[i].substring(8);
    }
  }

  if (window.location.protocol == "http:") {
    // If "sl" cookie is set, redirect to https.
    for (var i = 0; i < cookies.length; ++i) {
      if ((cookies[i].indexOf("sl=") == 0) && (cookies[i].length > 3)) {
        window.location.href = "https:" + window.location.href.substring(window.location.protocol.length);
        return;
      }
    }
  }

  // Get the new domain. For international domains such as
  // fr.linkedin.com, we convert it to www.linkedin.com
  var domain = "www.linkedin.c

### Soupify the Text

In [8]:
page = response.text

lxml is a library for processing XML and HTML in Python. We are parsing the data from txt to lxml.

In [9]:
# if needed: pip install beautifulsoup4 lxml or conda install beautifulsoup4 lxml
from bs4 import BeautifulSoup

soup = BeautifulSoup(page, "lxml")

In [10]:
print(soup)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html lang="en">
<head>
<meta content="text/html;charset=utf-8" http-equiv="Content-type"/>
<title>The Big Lebowski (1998) - Box Office Mojo</title>
<style type="text/css">
table.chart-wide { width: 100%; }
</style>
<meta content="the big lebowski, movie, film, box office, result, records, charts, revenue, opening weekend, gross, worldwide, overseas, foreign, news, reviews, articles, stories, story, analysis, revenue, release date, mpaa rating, genre, running time, length, budget, production budget, distributor, studio, gramercy, theatrical summary, theatrical, showtimes, tickets, show times, theaters, playing, weekend box office results, weekly box office, weekly box office, release summary, similar movies, box office mojo" name="keywords"/>
<meta content="The Big Lebowski summary of box office results, charts and release information and related links." name="description"/>
<link cha

### Prettify the Soup

A webpage can be thought of as a tree of elements, there is the 'body', which would contain a few 'divs' and each of those 'divs' can in turn contain 'divs' and other elements. A Soup object contains this tree. The prettify() method will turn a Beautiful Soup tree into a nicely formatted Unicode string, with each HTML/XML tag on its own line.

In [11]:
print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html lang="en">
 <head>
  <meta content="text/html;charset=utf-8" http-equiv="Content-type"/>
  <title>
   The Big Lebowski (1998) - Box Office Mojo
  </title>
  <style type="text/css">
   table.chart-wide { width: 100%; }
  </style>
  <meta content="the big lebowski, movie, film, box office, result, records, charts, revenue, opening weekend, gross, worldwide, overseas, foreign, news, reviews, articles, stories, story, analysis, revenue, release date, mpaa rating, genre, running time, length, budget, production budget, distributor, studio, gramercy, theatrical summary, theatrical, showtimes, tickets, show times, theaters, playing, weekend box office results, weekly box office, weekly box office, release summary, similar movies, box office mojo" name="keywords"/>
  <meta content="The Big Lebowski summary of box office results, charts and release information and related links." name="d

## Beautiful Soup - Find & Find_All

### `soup.find()`

* `soup.find()` is the most common function we will use from this package.  
* Let's try out some common variations of `soup.find()`

* `soup.find()` returns the first matched tag it finds.
* It searches the entire tree.

* Search for a type of tag by using the tag as a string argument ('body','div','p','a')

In [12]:
soup.find('a') # "a" tag is for hyperlink

<a href="/daily/chart/">Daily Box Office (Sun.)</a>

In [13]:
# Equivalently:
soup.a

<a href="/daily/chart/">Daily Box Office (Sun.)</a>

In [14]:
# Prettier:
print(soup.a.prettify())

<a href="/daily/chart/">
 Daily Box Office (Sun.)
</a>



Here's how you can find the next one.

In [15]:
soup.find('a').findNextSibling()

<a href="/weekend/chart/">Weekend Box Office (Jul. 5–7)</a>

### `soup.find_all()`

`soup.find_all()` returns a list of all matches

In [16]:
len(soup.find_all('a'))

100

In [17]:
for link in soup.find_all('a'): 
    print(link)

<a href="/daily/chart/">Daily Box Office (Sun.)</a>
<a href="/weekend/chart/">Weekend Box Office (Jul. 5–7)</a>
<a href="/movies/?id=spidermanhomecoming2.htm">#1 Movie: 'Spider-Man: Far from Home'</a>
<a href="http://www.imdb.com/showtimes/?ref_=mojo">Showtimes</a>
<a href="//bs.serving-sys.com/Serving/adServer.bs?cn=brd&amp;pli=1073982656&amp;Page=&amp;Pos=668343950" target="_blank">
<img border="0" height="90" src="//bs.serving-sys.com/Serving/adServer.bs?c=8&amp;cn=display&amp;pli=1073982656&amp;Page=&amp;Pos=668343950" width="728"/>
</a>
<a href="/"><img alt="Box Office Mojo" height="56" src="/img/misc/bom_logo1.png" width="245"/></a>
<a href="http://pro.imdb.com/signup/index.html?rf=mojo_nb_hm&amp;ref_=mojo_nb_hm" target="_blank">
<img alt="Get industry info at IMDbPro" height="20" src="/images/IMDbPro.png"/>
</a>
<a href="http://twitter.com/boxofficemojo" target="_blank">
<img alt="Follow us on Twitter" height="18" src="/images/glyphicons-social-32-twitter@2x.png"/>
</a>
<a href=

In [18]:
[link for link in soup.find_all('a') if 'joelcoen' in str(link)]

[<a href="/people/chart/?view=Director&amp;id=joelcoen.htm">Joel Coen</a>,
 <a href="/people/chart/?view=Writer&amp;id=joelcoen.htm">Joel Coen</a>]

## Beautiful Soup - More on Find

### `href` Example

In [ ]:
# retrieve the url from an anchor tag
soup.find('a')['href']

### `id` and `class` examples

* An attribute like id or class can be matched
* Example: 'mp_box_content' classes

In [ ]:
soup.find_all(id='top_links')

In [ ]:
for element in soup.find_all(class_='mp_box_content'):
    print(element, '\n')

## Beautiful Soup - Chaining Finds

All the fields in mp_box_content can be found by "chaining" a few `find_all` functions.

In [ ]:
# 'td' is for a cell in an HTML table
chain = [x.find_all('td') for x in soup.find_all(class_='mp_box_content')]

In [ ]:
# for the first mp_box_content find all td's
chain[0]

To extract just the value of interest:

In [ ]:
# Find the domestic gross. The '\xa0' represents a space in unicode
soup.find(class_='mp_box_content').find_all('td')[1].text

In [ ]:
# There are 2 td's the second one has the $17,451,873 and we remove the space character
soup.find(class_='mp_box_content').find_all('td')[1].text[1:] 

## Let's Practice Web Scraping!

### Items to scrape for each movie:

* Movie Title
* Domestic Total Gross
* Runtime
* MPAA Rating
* Release Date

### Movie Title

In [22]:
soup.find('title')

<title>The Big Lebowski (1998) - Box Office Mojo</title>

In [23]:
soup.find('title').text

'The Big Lebowski (1998) - Box Office Mojo'

In [24]:
title_string = soup.find('title').text
title_string

'The Big Lebowski (1998) - Box Office Mojo'

In [25]:
title_string.split('(')

['The Big Lebowski ', '1998) - Box Office Mojo']

In [26]:
# .strip() removes the white spaces at the beginning and end of the string
title = title_string.split('(')[0].strip() 
title

'The Big Lebowski'

### Domestic Total Gross

Let's try to find the exact text.

In [27]:
print(soup.find(text="Domestic Total Gross"))

None


`Text` does an exact match search, so we have to be careful.

In [28]:
print(soup.find(text="Domestic Total Gross: "))

Domestic Total Gross: 


What if we don't want to be careful? [Regular expressions](https://xkcd.com/208/) to the rescue!

We are going to talk a lot more about regular expressions in the next week or two, but there's a really powerful way to search for patterns in text. Today, we're going to use a very simple case, basically doing a "contains" instead of an "exact match".

In [29]:
import re
domestic_total_regex = re.compile('Domestic Total')
domestic_total_regex

re.compile(r'Domestic Total', re.UNICODE)

In [30]:
dtg_string = soup.find(text=domestic_total_regex)
dtg_string

'Domestic Total Gross: '

In [ ]:
dtg_string.findNextSibling()

We found the domestic total gross! Now let's strip it down and convert it to an integer.

In [ ]:
dtg = dtg_string.findNextSibling().text
print(dtg, type(dtg))

dtg = dtg.replace('$','').replace(',','')
print(dtg, type(dtg))

domestic_total_gross = int(dtg)
print(domestic_total_gross, type(domestic_total_gross))

### Runtime, MPAA Rating & Release Date

#### Step 1: Create Function to Identify Values

Let's make a function to scrape multiple things, assuming the value will always follow the field name.

In [ ]:
def get_movie_value(soup, field_name):
    
    '''Grab a value from boxofficemojo HTML
    
    Takes a string attribute of a movie on the page and returns the string in
    the next sibling object (the value for that attribute) or None if nothing is found.
    '''
    
    obj = soup.find(text=re.compile(field_name))
    
    if not obj: 
        return None
    
    # this works for most of the values
    next_sibling = obj.findNextSibling()
    
    if next_sibling:
        return next_sibling.text 
    else:
        return None

In [ ]:
# domestic total gross
dtg = get_movie_value(soup,'Domestic Total')
print(dtg)

In [ ]:
# runtime
runtime = get_movie_value(soup,'Runtime')
print(runtime)

In [ ]:
# rating
rating = get_movie_value(soup,'MPAA Rating')
print(rating)

In [ ]:
release_date = get_movie_value(soup,'Release Date')
print(release_date)

#### Step 2: Convert Values to Appropriate Data Types

In [ ]:
import dateutil.parser

def money_to_int(moneystring):
    moneystring = moneystring.replace('$', '').replace(',', '')
    return int(moneystring)

def runtime_to_minutes(runtimestring):
    runtime = runtimestring.split()
    try:
        minutes = int(runtime[0])*60 + int(runtime[2])
        return minutes
    except:
        return None

def to_date(datestring):
    date = dateutil.parser.parse(datestring)
    return date

#### Step 3: Apply the Conversions

In [ ]:
# Let's get these again and format them all in one swoop

raw_domestic_total_gross = get_movie_value(soup,'Domestic Total')
domestic_total_gross = money_to_int(raw_domestic_total_gross)

raw_runtime = get_movie_value(soup,'Runtime')
runtime = runtime_to_minutes(raw_runtime)

raw_release_date = get_movie_value(soup,'Release Date')
release_date = to_date(raw_release_date)

print(domestic_total_gross, runtime, release_date)
print(type(domestic_total_gross), type(runtime), type(release_date))

#### Step 4: Print It All Out

In [ ]:
from pprint import pprint # pretty print

headers = ['movie title', 'domestic total gross',
           'runtime (mins)', 'rating', 'release date']

movie_data = []
movie_dict = dict(zip(headers, [title,
                                domestic_total_gross,
                                runtime,
                                rating, 
                                release_date]))

movie_data.append(movie_dict)
pprint(movie_data)

## Table Scraping Example

### Step 1: Soupify the Website

Let's take a look at the foreign language page of Box Office Mojo. Let's say we wanted to pull all of the data from the main table on the page.

In [ ]:
url = 'http://www.boxofficemojo.com/genres/chart/?id=foreign.htm'

response = requests.get(url)
page = response.text

soup = BeautifulSoup(page,"lxml")

### Step 2: Find the Tables

In [ ]:
tables = soup.find_all("table")
tables

### Step 3: Pull Just the Rows

In [ ]:
rows = [row for row in tables[3].find_all('tr')] # tr tag is for rows

In [ ]:
# let's take a look at one row
rows[0]

In [ ]:
# let's take a look at one value in the row
rows[0].find_all('td')[1].find('a')['href']

### Step 4: Pull All Values

In [ ]:
rows[1].find_all('td')[1].find('a')['href']

In [ ]:
rows = rows[1:21] # let's just look at the first 20 rows for now
movies = {}

for row in rows:
    items = row.find_all('td')
    title = items[1].find('a')['href']
    movies[title] = [i.text for i in items[1:]]
    
list(movies.items())[0]

### Step 5: Pandas Alternative

In [ ]:
# you can also use pandas to read tables
import pandas as pd

url = 'http://www.boxofficemojo.com/genres/chart/?id=foreign.htm'

In [ ]:
tables = pd.read_html(url)

In [ ]:
tables[2]
# how can you fix the header?

In [ ]:
tables[2][0:5]

Conclusion: Beautiful Soup is powerful but it has many limitations. If a page needs interactions (like entering password) or if a page is not static, but dynamically generated, we can't use Soup. We will explore other tools for that.

One such tool for tomorrow: *Selenium*